<a href="https://colab.research.google.com/github/CAVASOL/aiffel_quest/blob/main/DL_quest/DL_prep/DL_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Backward propagation of errors

* 계산 결과와 정답의 오차를 구해 이 오차에 관여하는 값들의 가증치를 수정하여 오차가 작아지는 방향으로 일정 횟수를 반복해 수정하는 방법.
* 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 오류가 없는지 확인할 수 있음.

### 5-1. Computational Graph

* 단순한 계산에 집중하여 문제를 단순화.
* 중간 계산 결과를 모두 저장할 수 있음.
* 역전파를 통해 미분을 효율적으로 계산.

### 5-2. Chain rule

* 역전파(backpropagation) 알고리즘의 핵심.
* 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.
* 미분의 기본 원리 중 하나로, 함수가 다른 함수로 구성되어 있을 때 전체 함수의 도함수(미분)를 각 함수의 도함수의 곱으로 표현하는 규칙.

### 5-3. Backpropagation

* **기울기 계산**: 역전파는 손실 함수의 기울기(그레이디언트)를 효과적으로 계산하는 데 사용됩니다. 이는 모델의 가중치 및 편향을 업데이트하기 위해 필요한 정보로, 모델의 출력에서 입력 방향으로 오차를 전파하며 각 레이어에서 그레이디언트를 계산합니다.  


* **연쇄 법칙 활용**: 역전파는 연쇄 법칙(Chain Rule)을 기반으로 하며, 그레이디언트를 각 레이어로 전파하는 과정을 효율적으로 수행합니다. 이를 통해 많은 수의 레이어로 이루어진 딥 신경망에서도 그레이디언트를 계산할 수 있습니다.  


* **자동 미분**: 역전파는 자동 미분(automatic differentiation)의 핵심 요소입니다. 그레이디언트 계산을 자동으로 처리하므로 모델을 개발할 때 복잡한 미분식을 직접 유도하지 않아도 됩니다.  


* **신경망 학습**: 역전파는 신경망 모델의 가중치 및 편향을 학습하기 위해 사용됩니다. 손실 함수의 기울기를 최소화하는 방향으로 가중치를 조정하면 모델이 데이터를 더 잘 반영하도록 학습됩니다.  


* **비선형성 학습**: 역전파를 사용하면 다양한 활성화 함수(예: ReLU, 시그모이드, 소프트맥스)를 통해 비선형성을 학습할 수 있습니다. 이를 통해 신경망이 복잡한 함수 근사를 수행할 수 있습니다.  


* **반복 학습**: 역전파는 반복적인 최적화 알고리즘을 기반으로 하며, 데이터를 여러 번 사용하여 모델을 학습합니다. 이를 통해 모델이 데이터에 적응하고 예측 성능을 향상시킬 수 있습니다.  


* **병렬 및 분산 계산**: 역전파를 사용하면 그래프의 다양한 부분을 병렬로 계산하거나 분산 컴퓨팅을 활용하여 학습 속도를 향상시킬 수 있습니다.  

### 5-4. 계층 구현하기

**MulLayer**

In [ ]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y

        return out

    def backward(self, dout):
        dx = dout*self.y
        dy = dout*self.x

        return dx, dy

In [ ]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


**AddLayer**

In [ ]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx, dy

In [ ]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


### 5-5. ReLU / Sigmoid 활성화 함수 계층 구현하기

**ReLU**

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [ ]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [ ]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


**Sigmoid**

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

### 5-6. Affine / Softmax 계층 구현하기

**Affine**

* 입력 데이터에 가중치를 곱하고 편향을 더하는 선형 변환을 수행.
* Affine 연산은 딥러닝 모델의 히든 레이어에서 사용되며, 비선형성을 도입하기 위해 활성화 함수(예: ReLU)와 함께 사용.

In [ ]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

X.shape
W.shape
B.shape

Y = np.dot(X, W) + B

In [ ]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [ ]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [ ]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [ ]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

**Softmax**

* 입력 벡터를 받아 각 클래스에 대한 확률 분포로 변환.
* 주로 분류 문제에서 출력 레이어에서 사용.
* Softmax는 입력을 정규화하고, 각 클래스에 대한 예측 확률을 계산하여 이를 출력.
* 이 확률 값은 0과 1 사이에 있고, 모든 클래스의 확률의 합은 1.

In [ ]:
class SoftmasWithLoass:

    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

### 5-7. Backpropagation 구현하기

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:1.8380962847574696e-10
b1:1.0226736920347126e-09
W2:6.817680850200458e-08
b2:1.3701536088345234e-07


**오차역전파법을 사용한 학습 구현하기**

In [ ]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09736666666666667 0.0982
0.7969166666666667 0.8013
0.8770333333333333 0.8797
0.8973 0.9001
0.9079666666666667 0.9078
0.9138833333333334 0.9141
0.91915 0.9194
0.9235666666666666 0.9253
0.9279 0.9283
0.9314666666666667 0.9304
0.93405 0.9329
0.9364666666666667 0.9363
0.93795 0.9373
0.9411666666666667 0.9409
0.9430333333333333 0.9423
0.94595 0.9432
0.9468666666666666 0.945
